# SEIRX model runs for prevention measures investigation in universities

In [1]:
import pandas as pd
import numpy as np
from os.path import join
import os
import sys
import json
import seaborn as sns

from scseirx import analysis_functions as af

import data_creation_functions as dcf

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

from importlib import reload

* Vaccination fraction of [students in Austria](https://www.statistik.at/web_de/presse/126794.html)
* [Omicron incubation period](https://www.eurosurveillance.org/content/10.2807/1560-7917.ES.2021.26.50.2101147?TRACK=RSS): 3 days
* [Omicron SAR](https://www.medrxiv.org/content/10.1101/2021.12.27.21268278v1.full.pdf+html): 0.31

In [5]:
# number of runs in the ensemble (set to a low number so runs complete quickly
# for demonstration purposes)
N_runs = 28

# load the other screening parameters from file
screening_params = pd.read_csv(join('screening_params', 'measure_packages.csv'))

network_types = ['all'] # can also be 'TU' or 'NaWi'

params = [(N_runs, 
           row['u_mask'],
           row['l_mask'], 
           row['u_vaccination_ratio'],
           row['l_vaccination_ratio'],
           row['presence_fraction'],
           contact_network_type)
           for i, row in screening_params.iterrows()\
           for contact_network_type in network_types]

print('there are {} different parameter combinations'.format(len(params)))

there are 64 different parameter combinations


In [6]:
mode = 'intervention_screening'
with open('params/{}_measures.json'.format(mode), 'r') as fp:
    measures = json.load(fp)

In [11]:
%%time
reload(dcf)

mode = 'intervention_screening'
testing = False

# paths for data I/O
contact_network_src = '../../data/networks'

results = pd.DataFrame()
for p in params[0:1]:
    N_runs, u_mask, l_mask, u_vaccination_ratio,\
    l_vaccination_ratio, presence_fraction, contact_network_type = p
    
    dst = '../../data/simulation_results/testing/ensembles_{}_{}'\
        .format(mode, contact_network_type)
    
    dcf.run_ensemble(mode, N_runs, contact_network_src, contact_network_type, dst, 
            u_mask=u_mask, l_mask=l_mask,
            u_vaccination_ratio=u_vaccination_ratio,
            l_vaccination_ratio=l_vaccination_ratio,
            presence_fraction=presence_fraction, testing=testing)

running on T14s, using 14 cores


100%|██████████| 28/28 [14:59<00:00, 32.13s/it]  


CPU times: user 816 ms, sys: 176 ms, total: 992 ms
Wall time: 14min 59s
